# Query via RAG 

In [8]:
from dotenv import load_dotenv
from IPython.display import display, Markdown
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field
from langgraph.graph import END, StateGraph
from typing_extensions import TypedDict
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from typing import Tuple, List
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars

from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)

In [9]:
# load the LLM and neo4j graph
load_dotenv()

llm = ChatOpenAI(temperature=0, model="gpt-4o")

graph = Neo4jGraph()

In [10]:
vector_index = Neo4jVector.from_existing_graph(
    OpenAIEmbeddings(),
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)


class Entities(BaseModel):
    """Identifying information about entities."""

    names: List[str] = Field(
        ...,
        description="All the person, object, location, or event entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are extracting person, object, location, or event entities from the text.",
        ),
        (
            "human",
            "Use the given format to extract information from the following "
            "input: {question}",
        ),
    ]
)

entity_chain = prompt | llm.with_structured_output(Entities)

def generate_full_text_query(input: str) -> str:
    """
    Generate a full-text search query for a given input string.

    This function constructs a query string suitable for a full-text search.
    It processes the input string by splitting it into words and appending a
    similarity threshold (~2 changed characters) to each word, then combines
    them using the AND operator. Useful for mapping entities from user questions
    to database values, and allows for some misspelings.
    """
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    return full_text_query.strip()

# Fulltext index query
def structured_retriever(question: str) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    entities = entity_chain.invoke({"question": question})
    for entity in entities.names:
        response = graph.query(
            # Fulltext index query - call entity name "entityIdIndex"
            """
            CALL db.index.fulltext.queryNodes('entityIdIndex', $query, {limit: 2})
            YIELD node, score
            CALL {
              WITH node
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              WITH node
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' + node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        print(response)
        result += "\n".join([el['output'] for el in response])
    print(result)

    return result

def retriever(question: str):
    structured_data = structured_retriever(question)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
        {structured_data}
        Unstructured data:
        {"#Document ". join(unstructured_data)}
    """
    return final_data

_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be as elaborate as possible.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    | llm
    | StrOutputParser()
)

def invoke_chain(question: str, chat_history):
    print("invoke chain called")
    graph.query(
        "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")
    print("fulltext index created")
    if chat_history:
        return chain.invoke(
            {
                "question": question,
                "chat_history": chat_history
            }
        )
    else:
        return chain.invoke(
            {
                "question": question,
            }
        )
    

############################################################# Graph State #############################################################
class GraphState(TypedDict): # might not actually need langgraph to organise the workflow since only one node "generate"
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        search_query: revised question for web search
        context: web_search result
    """
    question : str
    generation : str
    search_query : str
    context : str

# Node - Generate

def generate(state):
    """
    Generate answer

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    
    print("Step: Generating Final Response")
    question = state["question"]

    # Answer Generation
    generation = invoke_chain(question, None)
    return {"generation": generation}

# Conditional Edge, Routing

def route_question(state):
    """
    route question to web search or generation.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("Step: Routing Query")
    question = state['question']
    structured_data = structured_retriever(question)
    
    if len(structured_data) != 0:
        print("Step: Context Found, Routing to Generation")
        return "generate"
    elif len(structured_data) == 0: # Might need to change do deal with a case where context is not found
        print("Step: Context Not Found, Returning to Generation")
        return "generate"
    
def build_workflow():
    """
    Build the workflow for the graph
    """
    # Build the nodes
    workflow = StateGraph(GraphState)
    workflow.add_node("generate", generate)

    # Build the edges
    workflow.set_conditional_entry_point(
        route_question,
        {
            "generate": "generate",
        },
    )
    workflow.add_edge("generate", END)

    # Compile the workflow
    local_agent = workflow.compile()

    return local_agent

def run_agent(query, local_agent):
    output = local_agent.invoke({"question": query})
    print("=======")
    display(Markdown(output["generation"]))

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: line: 1, column: 70, offset: 69} for query: "UNWIND $data AS row MATCH (n:`Document`) WHERE elementId(n) = row.id CALL db.create.setVectorProperty(n, 'embedding', row.embedding) YIELD node RETURN count(*)"


Searching KG by "keyword"

In [9]:
# Test it out!
local_agent = build_workflow()
test_query = "Give me a short description of Utopolis. Please include the name of the mayor."
run_agent(test_query, local_agent)

Step: Routing Query
Step: Context Not Found, Returning to Generation
Step: Generating Final Response
invoke chain called
fulltext index created


Utopolis is a forward-thinking city dedicated to sustainability and high quality of life for its residents. Under the visionary leadership of Mayor Anya, Utopolis has made significant strides in various areas, including green energy, waste management, and public transportation. The city boasts an impressive recycling rate of 80% and has reduced car usage by 50% through its modern metro system, electric buses, and bike-sharing programs. Green spaces cover 30% of the city, enhancing air quality and providing recreational opportunities. Utopolis's education system emphasizes critical thinking and sustainability, with 95% of high school graduates pursuing higher education. The city collaborates globally, sharing best practices and supporting sustainable projects. Residents enjoy a high quality of life, with a 90% satisfaction rating, making Utopolis a model for sustainable urban living.

Searching KG by "entityIdIndex"

In [11]:
# Test it out!
local_agent = build_workflow()
test_query = "Give me a short description of Utopolis. Please include the name of the mayor."
run_agent(test_query, local_agent)

Step: Routing Query
Step: Context Found, Routing to Generation
Step: Generating Final Response
invoke chain called
fulltext index created


Utopolis is a forward-thinking city dedicated to sustainability and livability. Under the visionary leadership of Mayor Anya, Utopolis has undergone a transformative journey, prioritizing renewable energy sources, reducing reliance on fossil fuels, and mitigating greenhouse gas emissions. The city boasts an impressive recycling rate of 80% and a public transportation system that has cut car usage by 50%. Green spaces, including parks and urban gardens, cover 30% of the city, enhancing air quality and providing recreational opportunities. Utopolis's education system emphasizes critical thinking and sustainability, with 95% of high school graduates pursuing higher education. The city actively involves its citizens in planning and implementing urban initiatives, fostering a strong sense of community and ownership. Utopolis stands as a model of how sustainability can create a harmonious and thriving urban environment.

Perform testing on below cell instead

In [15]:
# Test it out!
local_agent = build_workflow()
test_query = "What are some green projects in Ecodora?"
run_agent(test_query, local_agent)

Step: Routing Query
Step: Context Found, Routing to Generation
Step: Generating Final Response
invoke chain called
fulltext index created


Ecodora has implemented a variety of green projects that contribute to its sustainability and enhance the quality of life for its residents. Here are some of the key green projects in Ecodora:

1. **Eco-Transit Revolution**:
   - **Eco-Metro**: Ecodora has invested in a modern metro system called the "Eco-Metro," which includes the construction of 5 new metro lines serving over 2 million passengers daily. This initiative significantly reduces the reliance on cars and helps lower carbon emissions.
   - **E-Ride**: The city introduced 300 electric buses branded as "E-Ride," which have contributed to a 20% reduction in carbon emissions.
   - **Wheel Share**: Ecodora has established over 700 bike-sharing stations with 15,000 bicycles available for rent, promoting cycling as a sustainable mode of transportation.

2. **Ecodora Oasis Initiative**:
   - **Central Eco Haven**: A 75-acre central park featuring hiking trails, playgrounds, and a serene lake, providing a green space for recreation and relaxation.
   - **Green Canopy Gardens**: More than 150 buildings in Ecodora showcase green roofs, totaling 75 acres of rooftop greenery, which help enhance air quality and foster biodiversity.
   - **Neighborhood Gardens**: The city has created 30 neighborhood gardens, offering gardening plots for residents and promoting community engagement in green activities.

3. **Ecodora, the Energy Champion**:
   - **Eco-Harmony 3.0**: New building codes mandate the use of energy-efficient appliances and enhanced thermal insulation to reduce energy consumption.
   - **Retrofit Renaissance**: Over 500 existing buildings have undergone energy-saving renovations, improving their energy efficiency.
   - **Solar Splendor**: Solar panels have been installed on more than 1,500 government buildings, harnessing renewable energy and reducing the city's carbon footprint.

4. **Eco-Living Paradise**:
   - **Recycle with Care**: Over 85% of households actively engage in Ecodora's recycling program, significantly reducing waste.
   - **Composting Crusaders**: The city has placed 400 community composting bins throughout neighborhoods, encouraging residents to compost organic waste.
   - **Waste-to-Biogas Facility**: An innovative plant that transforms organic waste into renewable energy, contributing to a circular economy.

5. **Ecodoran Civic Engagement**:
   - **Ecodora Community Forums**: Regular public meetings held in each district to gather citizen input and involve residents in decision-making processes.
   - **Ecodora Citizen Councils**: Fifteen councils have been formed to advise on specific policy areas and projects, ensuring that community voices are heard.
   - **My Ecodora Portal**: An online platform that allows residents to submit ideas, report issues, and track the progress of community initiatives, enhancing transparency and engagement.

These projects collectively demonstrate Ecodora's commitment to sustainability, community involvement, and innovative urban planning, making it a model city for green living.

In [7]:
# Test it out!
local_agent = build_workflow()
test_query = "What are some green projects in Ecodora?"
run_agent(test_query, local_agent)

Step: Routing Query
[{'output': 'Ecodora - PRIORITIZES -> Education System'}, {'output': 'Ecodora - IMPLEMENTED -> Recycle With Care'}, {'output': 'Ecodora - IMPLEMENTED -> Public Transportation System'}, {'output': 'Ecodora - ENGAGES_IN -> Global Collaborations'}, {'output': 'Ecodora - HARNESSES -> Renewable Energy Sources'}, {'output': 'Ecodora - BOASTS -> Recycling Rate'}, {'output': 'Ecodora - REDUCED -> Car Usage'}, {'output': 'Ecodora - ENCOMPASSES -> Parks And Green Spaces'}, {'output': 'Ecodora - ENJOYS -> Quality Of Life'}, {'output': 'Ecodora - INVESTED_IN -> Eco-Metro'}, {'output': 'Ecodora - INVESTED_IN -> E-Ride'}, {'output': 'Ecodora - INVESTED_IN -> Wheel Share'}, {'output': 'Ecodora - ESTABLISHED -> Central Eco Haven'}, {'output': 'Ecodora - ESTABLISHED -> Green Canopy Gardens'}, {'output': 'Ecodora - ESTABLISHED -> Neighborhood Gardens'}, {'output': 'Ecodora - EMBRACED -> Eco-Harmony 3.0'}, {'output': 'Ecodora - EMBRACED -> Retrofit Renaissance'}, {'output': 'Ecodora -

Ecodora has implemented a variety of green projects that significantly contribute to its sustainability and quality of life. Here are some of the key initiatives:

1. **Eco-Transit Revolution**:
   - **Eco-Metro**: Ecodora has invested in a modern metro system called the "Eco-Metro," which includes the construction of 5 new metro lines serving over 2 million passengers daily. This initiative has greatly reduced car usage and carbon emissions.
   - **E-Ride**: The city introduced 300 electric buses branded as "E-Ride," which have reduced carbon emissions by 20%.
   - **Wheel Share**: Ecodora has established over 700 bike-sharing stations with 15,000 bicycles available for rent, promoting eco-friendly transportation.

2. **Ecodora Oasis Initiative**:
   - **Central Eco Haven**: A 75-acre central park featuring hiking trails, playgrounds, and a serene lake, providing a green oasis for residents.
   - **Green Canopy Gardens**: More than 150 buildings in Ecodora showcase green roofs, totaling 75 acres of rooftop greenery, which enhance air quality and foster biodiversity.
   - **Neighborhood Gardens**: The city has created 30 neighborhood gardens, offering gardening plots for residents and promoting community engagement.

3. **Ecodora, the Energy Champion**:
   - **Eco-Harmony 3.0**: New building codes mandate the use of energy-efficient appliances and enhanced thermal insulation to reduce energy consumption.
   - **Retrofit Renaissance**: Over 500 existing buildings have undergone energy-saving renovations.
   - **Solar Splendor**: Solar panels have been installed on more than 1,500 government buildings, harnessing renewable energy sources.

4. **Eco-Living Paradise**:
   - **Recycle with Care**: Over 85% of households actively engage in the city's recycling program, significantly reducing waste.
   - **Composting Crusaders**: Ecodora has placed 400 community composting bins throughout neighborhoods to encourage composting.
   - **Waste-to-Biogas Facility**: An innovative plant transforms organic waste into renewable energy, contributing to a closed-loop waste management system.

5. **Ecodoran Civic Engagement**:
   - **Ecodora Community Forums**: Regular public meetings are held in each district to gather citizen input on various projects and policies.
   - **Ecodora Citizen Councils**: Fifteen councils have been formed to advise on specific policy areas and projects, ensuring community involvement in decision-making.
   - **My Ecodora Portal**: An online platform allows residents to submit ideas, report issues, and track the progress of community initiatives.

These projects collectively enhance Ecodora's sustainability, improve the quality of life for its residents, and foster a strong sense of community and environmental stewardship.